### libs

In [2]:
import time
import base64
from datetime import datetime, timedelta
import json
import logging
import requests
import streamlit as st
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery
from cryptography.fernet import Fernet
import json
from typing import List, Any, Dict
from tqdm import tqdm
import numpy as np
import os
import toml

### setup

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

secrets_path = os.path.join(os.path.dirname(os.path.abspath('calculadora.ipynb')), '..', '.streamlit', 'secrets.toml')

with open(secrets_path) as secrets_file:
    secrets = toml.load(secrets_file)

credentials = service_account.Credentials.from_service_account_info(secrets["gcp_service_account"])

client = bigquery.Client(credentials=credentials)

### bigquery

In [4]:
def send_bigquery(df: pd.DataFrame, table_id: str, schema: list):
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition="WRITE_TRUNCATE",
    )

    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()
    table = client.get_table(table_id)
    print(
        "Loaded {} rows and {} columns to {} in {}".format(
            table.num_rows, len(table.schema), table_id, datetime.now()
        )
    )

    print("Send to biquery !")


def query_bigquery(sql_query: str) -> pd.DataFrame:
    query_job = client.query(sql_query)

    result_query = query_job.result()

    df = pd.DataFrame(
        data=[row.values() for row in result_query],
        columns=[field.name for field in result_query.schema],
    )

    return df

### criptografia

In [5]:
def encrypt_password(password, key):
    fernet = Fernet(key)
    encrypted_password = fernet.encrypt(password.encode())
    return encrypted_password


def decrypt_password(encrypted_password, key):
    fernet = Fernet(key)
    decrypted_password = fernet.decrypt(encrypted_password).decode()
    return decrypted_password

### api bling

In [6]:
class api_bling:
    def __init__(self):
        self.cache = {}
        self._401_count = 0

    def get(self, url, loja: str):
        try:
            titulo_loja = "".join(loja.split()).upper()
            acess_token = self._access_token(titulo_loja)

            headers = {
                "Accept": "application/json",
                "Authorization": f"Bearer {acess_token}",
            }

            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                return response.json()
            elif response.status_code == 401:
                if self._401_count >= 2:
                    response.raise_for_status()

                self._401_count += 1
                self.cache.clear()
                return self.get(url, loja)
            elif response.status_code == 429:
                time.sleep(1)
                return self.get(url, loja)
            else:
                response.raise_for_status()
        except Exception as e:
            print("Ocorreu um erro:", e)
            return "error"

    def post(self, url, body, loja: str):
        try:
            titulo_loja = "".join(loja.split()).upper()
            acess_token = self._access_token(titulo_loja)

            headers = {
                "Content-Type": "application/json",
                "Accept": "application/json",
                "Authorization": f"Bearer {acess_token}",
            }
            payload = json.dumps(body)
            response = requests.post(url, headers=headers, data=payload)

            if response.status_code == 200:
                return response.json()
            elif response.status_code == 401:
                if self._401_count >= 2:
                    response.raise_for_status()

                self._401_count += 1
                self.cache.clear()
                return self.post(url, loja)
            elif response.status_code == 429:
                time.sleep(1)
                return self.post(url, loja)
            else:
                response.raise_for_status()
        except Exception as e:
            print("Ocorreu um erro:", e)
            return "error"

    def _oauth_refresh(self, df_credenciais: pd.DataFrame, loja: str) -> str:
        chave_criptografia = str(st.secrets["chave_criptografia"]).encode()
        bling_client_id = st.secrets[f"BLING_CLIENT_ID_{loja}"]
        bling_client_secret = st.secrets[f"BLING_CLIENT_SECRET_{loja}"]

        refresh_token = decrypt_password(
            str(
                df_credenciais["valor"]
                .loc[
                    (df_credenciais["titulo"] == "refresh_token")
                    & (df_credenciais["loja"] == f"BLING_{loja}")
                ]
                .values[0]
            ),
            chave_criptografia,
        )

        credentialbs4 = f"{bling_client_id}:{bling_client_secret}"
        credentialbs4 = credentialbs4.encode("ascii")
        credentialbs4 = base64.b64encode(credentialbs4)
        credentialbs4 = bytes.decode(credentialbs4)

        header = {"Accept": "1.0", "Authorization": f"Basic {credentialbs4}"}
        dice = {"grant_type": "refresh_token", "refresh_token": refresh_token}

        api = requests.post(
            "https://www.bling.com.br/Api/v3/oauth/token", headers=header, json=dice
        )

        situationStatusCode = api.status_code
        api = api.json()

        if situationStatusCode == 400:
            print(f"Request failed. code: {situationStatusCode}")

        df_credenciais.loc[
            (df_credenciais["loja"] == f"BLING_{loja}")
            & (df_credenciais["titulo"] == "access_token"),
            "valor",
        ] = encrypt_password(api["access_token"], chave_criptografia)

        df_credenciais.loc[
            (df_credenciais["loja"] == f"BLING_{loja}")
            & (df_credenciais["titulo"] == "access_token"),
            "validade",
        ] = str(datetime.now())

        df_credenciais.loc[
            (df_credenciais["loja"] == f"BLING_{loja}")
            & (df_credenciais["titulo"] == "refresh_token"),
            "valor",
        ] = encrypt_password(api["refresh_token"], chave_criptografia)

        df_credenciais.loc[
            (df_credenciais["loja"] == f"BLING_{loja}")
            & (df_credenciais["titulo"] == "refresh_token"),
            "validade",
        ] = str(datetime.now())

        schema = [
            bigquery.SchemaField("loja", "STRING"),
            bigquery.SchemaField("titulo", "STRING"),
            bigquery.SchemaField("validade", "STRING"),
            bigquery.SchemaField("valor", "STRING"),
        ]
        table_id = f"integracao-414415.data_ptl.credenciais"

        send_bigquery(df_credenciais, table_id, schema)

        return api["access_token"]

    def _validade_access_token(self, df_credenciais: pd.DataFrame, loja: str) -> str:

        data_atualizacao = datetime.strptime(
            df_credenciais["validade"]
            .loc[
                (df_credenciais["titulo"] == "access_token")
                & (df_credenciais["loja"] == f"BLING_{loja}")
            ]
            .values[0],
            "%Y-%m-%d %H:%M:%S.%f",
        )

        data_limite = data_atualizacao + timedelta(hours=6)

        if datetime.now() > data_limite or self._401_count >= 2:
            return self._oauth_refresh(df_credenciais, loja)

        return decrypt_password(
            df_credenciais["valor"]
            .loc[
                (df_credenciais["titulo"] == "access_token")
                & (df_credenciais["loja"] == f"BLING_{loja}")
            ]
            .values[0],
            str(st.secrets["chave_criptografia"]).encode(),
        )

    def _access_token(self, loja: str) -> str:
        if loja in self.cache:
            return self.cache[loja]

        df_credenciais = query_bigquery(
            "SELECT * FROM `integracao-414415.data_ptl.credenciais`"
        )

        self.cache[loja] = self._validade_access_token(df_credenciais, loja)

        return self.cache[loja]


### requisitando saldo de estoque

In [14]:
def get_saldo_estoque(loja: str, lista_produtos: List[str]) -> List[Dict[str, Any]]:
    # with st.spinner(f"Atualizando saldo de estoque {loja}..."):
    api = api_bling()
    dados_estoque = []

    for i in tqdm(range(0, len(lista_produtos), 5)):
        time.sleep(0.1)
        selecao = "&idsProdutos[]=" + "&idsProdutos[]=".join(
            lista_produtos[i : i + 5]
        )

        try:
            response = api.get(
                f"https://www.bling.com.br/Api/v3/estoques/saldos?{selecao}",
                loja,
            )

            if response == "error" or response.get("data", []) == []:
                next(iter(lista_produtos))

            for produto in response.get("data", []):
                id_produto: str = str(produto.get("produto", {}).get("id", ""))
                saldo_fisico_total: float = float(
                    produto.get("saldoFisicoTotal", 0)
                )

                saldo_estoques = {
                    2816599510: 0,
                    11910799658: 0,
                    14887642988: 0,
                    9738790725: 0,
                    14197230585: 0,
                    14886665514: 0,
                }

                for deposito in produto.get("depositos", []):
                    saldo_estoques[deposito.get("id", 0)] = float(
                        deposito.get("saldoFisico", 0)
                    )

                dados_estoque.append(
                    {
                        "id": id_produto,
                        "saldo_fisico_total": saldo_fisico_total,
                        "estoque_matriz": saldo_estoques[2816599510],
                        "full_amazon": saldo_estoques[11910799658],
                        "full_mgl_proteloja": saldo_estoques[14887642988],
                        "full_ml_proteloja": saldo_estoques[9738790725],
                        "full_ml_vendolandia": saldo_estoques[14197230585],
                        "full_ml_vendolandia2": saldo_estoques[14886665514],
                        "loja": loja,
                    }
                )

        except Exception as e:
            print("Error get_fornecedores_bling /fornecedores: ", e)

    return dados_estoque

### atualizando saldo de estoque

In [15]:
def atualizar_saldo_estoque() -> List[Dict[str, Any]]:
    lista_id_produtos = query_bigquery(
        """ 
            SELECT 
                CAST(id AS STRING) AS id,
                estoque_maximo,
                sku,
                loja,
                variacao,
                estrutura,
                nome
            FROM 
                `integracao-414415.data_ptl.produtos_bling_proteloja`
            WHERE 
                variacao != '{}'
                AND estrutura = "{'tipoEstoque': '', 'lancamentoEstoque': '', 'componentes': []}"
                AND UPPER(nome) NOT LIKE '%CONSUMIVEL%'
        """
    )
    
    df_saldo_estoque_bling_proteloja = pd.DataFrame(
        get_saldo_estoque(
            "proteloja", lista_id_produtos[lista_id_produtos["loja"] == "proteloja"]["id"].array
        )
    )
        
    df_saldo_estoque_bling_vendolandia = pd.DataFrame(
        get_saldo_estoque(
            "vendolandia",
            lista_id_produtos[lista_id_produtos["loja"] == "vendolandia"]["id"].array,
        )
    ) 

    df_concatenado = pd.concat(
        [df_saldo_estoque_bling_proteloja, df_saldo_estoque_bling_vendolandia], ignore_index=True
    )     

    if not df_concatenado.empty:
        df_concatenado.fillna(0, inplace=True)

        schema = [
            bigquery.SchemaField("id", "STRING"),
            bigquery.SchemaField("saldo_fisico_total", "FLOAT"),
            bigquery.SchemaField("estoque_matriz", "FLOAT"),
            bigquery.SchemaField("full_amazon", "FLOAT"),
            bigquery.SchemaField("full_mgl_proteloja", "FLOAT"),
            bigquery.SchemaField("full_ml_proteloja", "FLOAT"),
            bigquery.SchemaField("full_ml_vendolandia", "FLOAT"),
            bigquery.SchemaField("full_ml_vendolandia2", "FLOAT"),
            bigquery.SchemaField("loja", "STRING"),
        ]
        table_id = f"integracao-414415.data_ptl.saldo_estoque_bling"

        send_bigquery(df_concatenado, table_id, schema)

### requisitando pedidos de compra

In [16]:
def get_compra_bling(loja: str) -> List[Dict[str, Any]]:
    # with st.spinner(f"Requisitando pedidos de compra {loja}..."):
    api = api_bling()
    id_pedidos = []
    dados_pedidos = []
    page = 1

    today = datetime.now().date()
    ontem = today - timedelta(days=90)

    data_anterior = ontem.strftime("%Y-%m-%d")
    data_posterior = today.strftime("%Y-%m-%d")

    while True:
        try:
            response = api.get(
                f"https://www.bling.com.br/Api/v3/pedidos/compras?pagina={page}&limite=100&idSituacao=28&dataInicial={data_anterior}&dataFinal={data_posterior}",
                loja,
            )

            if response == "error" or response["data"] == []:
                break

            for i in response["data"]:
                id_pedidos.append(i.get("id", ""))

            page += 1
        except Exception as e:
            print("Error get_pedidos_compra_bling /compras: ", e)

    if id_pedidos == []:
        return []

    for pedido in tqdm(id_pedidos):
        if pedido != "" and pedido != None:
            try:
                response = api.get(
                    f"https://www.bling.com.br/Api/v3/pedidos/compras/{pedido}",
                    loja,
                )

                if response == []:
                    continue

                pedido: list = response.get("data", [])

                for item in pedido.get("itens", {}):
                    id_pedido: str = str(pedido.get("id", ""))
                    numero_pedido: str = str(pedido.get("numero", ""))
                    data: datetime = (
                        datetime.strptime(str(pedido["data"]), "%Y-%m-%d")
                        if pedido["data"] and pedido["data"] != "0000-00-00"
                        else None
                    )
                    data_prevista: datetime = (
                        datetime.strptime(str(pedido["dataPrevista"]), "%Y-%m-%d")
                        if pedido["data"] and pedido["dataPrevista"] != "0000-00-00"
                        else None
                    )
                    total: float = float(pedido.get("total", 0))
                    fornecedor: str = str(pedido.get("fornecedor", {}).get("id", ""))
                    situacao: str = str(pedido.get("situacao", {}).get("valor", ""))
                    nome_produto: str = str(item.get("descricao", ""))
                    valor_produto: float = float(item.get("valor", 0))
                    quantidade: float = float(item.get("quantidade", 0))
                    id_produto: str = str(item.get("produto", {}).get("id", ""))
                    sku: str = str(item.get("produto", {}).get("codigo", ""))

                    dados_pedidos.append(
                        {
                            "id_pedido": id_pedido,
                            "numero_pedido": numero_pedido,
                            "data": data,
                            "data_prevista": data_prevista,
                            "total": total,
                            "fornecedor": fornecedor,
                            "situacao": situacao,
                            "nome_produto": nome_produto,
                            "valor_produto": valor_produto,
                            "quantidade": quantidade,
                            "id_produto": id_produto,
                            "sku": sku,
                            "loja": loja,
                        }
                    )

            except Exception as e:
                print("Error get_pedidos_compra_bling /compras/pedido: ", e)

    return dados_pedidos


### atualizar compras

In [17]:
def atualizar_compras_bling():
    df_compras_proteloja = pd.DataFrame(get_compra_bling("proteloja"))
    df_compras_vendolandia = pd.DataFrame(get_compra_bling("vendolandia"))

    df_compras = pd.concat(
        [df_compras_proteloja, df_compras_vendolandia], ignore_index=True
    )

    if df_compras.empty == False:
        schema = [
            bigquery.SchemaField("id_pedido", "STRING"),
            bigquery.SchemaField("numero_pedido", "STRING"),
            bigquery.SchemaField("data", "DATE"),
            bigquery.SchemaField("data_prevista", "DATE"),
            bigquery.SchemaField("total", "FLOAT"),
            bigquery.SchemaField("fornecedor", "STRING"),
            bigquery.SchemaField("situacao", "STRING"),
            bigquery.SchemaField("nome_produto", "STRING"),
            bigquery.SchemaField("valor_produto", "FLOAT"),
            bigquery.SchemaField("quantidade", "FLOAT"),
            bigquery.SchemaField("id_produto", "STRING"),
            bigquery.SchemaField("sku", "STRING"),
            bigquery.SchemaField("loja", "STRING"),
        ]
        table_id = f"integracao-414415.data_ptl.compras_bling"

        send_bigquery(df_compras, table_id, schema)

    df_compras["data_prevista"] = pd.to_datetime(
        df_compras["data_prevista"].fillna(''), errors="coerce"
    )

    #para cada id_produto, selecionar somente as linhas com datas minimas, e os valores da coluna quantidade e data prevista
    df_min_dates = df_compras.groupby("id_produto")["data_prevista"].min().reset_index()
    df_min = pd.merge(df_min_dates, df_compras, on=["id_produto", "data_prevista"])
    df_resultado_compras = df_min[["id_produto", "quantidade", "data_prevista"]]
    df_resultado_compras["quantidade"] = df_resultado_compras["quantidade"].fillna(0)


### atualizando dados

In [ ]:
atualizar_saldo_estoque()
atualizar_compras_bling()

### código

- tempo entrega full
- tempo entrega fornecedor
- periodo vendas analisado
- quantidade de vendas diaria
- quantidade de vendas total
- sku
- titulo
- custo unitario
- estoque atual
- estoque de segurança (25 dias)
- tempo de entrega total
---
- id compra
- nome fornecedor
- data da compra
- quantidade comprada

- data entrega fornecedor
- data entrega full
- quanto comprar 
- estoque futuro
- tempo próxima entrega fornecedor

In [5]:
df = query_bigquery(
"""
  WITH
    dias_entrega_fornecedor AS (
      SELECT
        sku,
        dias_entrega_fornecedor,
        dias_entrega_full
      FROM
        `integracao-414415.data_ptl.dia_entrega_fornecedor`
    ),
    produtos AS (
      SELECT
        estoque_maximo,
        id,
        nome,
        sku
      FROM
        `integracao-414415.data_ptl.produtos_bling_proteloja`
    ),
    fornecedores AS (
      SELECT
        id_produto,
        id_fornecedor,
        nome_fornecedor,
        custo,
      FROM
        `integracao-414415.data_ptl.fornecedores_bling_proteloja`
    ),
    estoque AS (
      SELECT
        id,
        saldo_fisico_total
      FROM 
        `integracao-414415.data_ptl.saldo_estoque_bling`
    )
  SELECT
    d.dias_entrega_fornecedor,
    d.dias_entrega_full,
    p.estoque_maximo,
    c.sku,
    p.nome,
    f.custo,
    e.saldo_fisico_total,
    c.numero_pedido,
    f.nome_fornecedor,
    c.data,
    c.data_prevista,
    c.quantidade,
  FROM
    `integracao-414415.data_ptl.compras_bling` c
    JOIN fornecedores f ON c.fornecedor = f.id_fornecedor
    AND c.id_produto = f.id_produto
    JOIN dias_entrega_fornecedor d ON c.sku = d.sku
    JOIN produtos p ON c.id_produto = p.id
    JOIN estoque e ON c.id_produto = e.id
  WHERE
    c.sku = "101104-V"
"""
)

In [6]:
df.head()

,dias_entrega_fornecedor,dias_entrega_full,estoque_maximo,sku,nome,custo,saldo_fisico_total,numero_pedido,nome_fornecedor,data,data_prevista,quantidade
0,30.0,8.0,28.0,101104-V,TRAVA QUEDAS FITA RETRÁTIL MEDBLOC 6M Tamanho:...,539.0,14.0,2945,DELTA PLUS BRASIL INDUSTRIA E COMERCIO DE EPI ...,2024-08-14,2024-10-18,18.0
1,30.0,8.0,28.0,101104-V,TRAVA QUEDAS FITA RETRÁTIL MEDBLOC 6M Tamanho:...,539.0,14.0,2946,DELTA PLUS BRASIL INDUSTRIA E COMERCIO DE EPI ...,2024-08-14,2024-11-01,48.0
2,30.0,8.0,28.0,101104-V,TRAVA QUEDAS FITA RETRÁTIL MEDBLOC 6M Tamanho:...,539.0,14.0,3019,DELTA PLUS BRASIL INDUSTRIA E COMERCIO DE EPI ...,2024-09-26,None,0.0


In [7]:
tempo_entrega_fornecedor = df["dias_entrega_fornecedor"][0]
tempo_entrega_full =  df["dias_entrega_full"][0]
tempo_entrega_total = tempo_entrega_fornecedor + tempo_entrega_full

sku = df["sku"][0]  
nome_produto = df["nome"][0]

estoque_atual = df["saldo_fisico_total"][0]


media_venda_diaria = df["estoque_maximo"][0] / 50
estoque_seguranca = media_venda_diaria * 25
estoque_maximo = estoque_seguranca + (media_venda_diaria * tempo_entrega_total)

# id_compra, nome_fornecedor, data_compra, quantidade_comprada, data_entrega_fornecedor, data_entrega_full,
# quanto_comprar, estoque_futuro, 


df_tabela_compras = df[["numero_pedido", "nome_fornecedor", "data", "quantidade", "data_prevista"]]
df_tabela_compras

# se a data prevista for none, calcular a data mais tempo_entrega_fornecedor

# calcular o quanto comprar e estoque futuro 

,numero_pedido,nome_fornecedor,data,quantidade,data_prevista
0,2945,DELTA PLUS BRASIL INDUSTRIA E COMERCIO DE EPI ...,2024-08-14,18.0,2024-10-18
1,2946,DELTA PLUS BRASIL INDUSTRIA E COMERCIO DE EPI ...,2024-08-14,48.0,2024-11-01
2,3019,DELTA PLUS BRASIL INDUSTRIA E COMERCIO DE EPI ...,2024-09-26,0.0,None
